# Inference

Load model, weights and do some inference!

In [1]:
# imports torch
import torch
import torchvision
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Subset

# stats
import glob
import pandas as pd
import numpy as np

In [2]:
model_path = '../models/resnet50_14_04_21.pth'

# model 
model = torchvision.models.resnet50()
model.conv1 = nn.Conv2d(4, 64, (7, 7), 
                        stride=(2,2), 
                        padding=(3,3), 
                        bias=False)

# output features to 3 classification
model.fc = torch.nn.Sequential(
    torch.nn.Linear(
        in_features=2048,
        out_features=19
    ),
    torch.nn.Sigmoid()
)

In [3]:

# path for saving models
PATH = '../models/resnet50_14_04_21.pth'
model.load_state_dict(torch.load(PATH))
model.eval()

ResNet(
  (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
with torch.no_grad():
    # do inference

SyntaxError: unexpected EOF while parsing (<ipython-input-4-4d3778c754b9>, line 2)

In [3]:
list_masked = [ name.split('_predictedmask.png')[0].split('mask/')[1].strip() for\
      name in glob.glob('../input/image_subset_example/mask/*.png')]

In [9]:
train = pd.read_csv('../input/image_subset_example/example.csv')

In [11]:
train.iloc[0, 3:]

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    1.0
Name: 0, dtype: object

In [5]:
examples = train[train.ID.isin(list_masked)==True].reset_index(drop=True)

In [6]:
# Transform string label to 19 one-hot columns
cols = [str(i) for i in list(range(19))]
examples[cols] = np.nan

# Apply mapping
def encoder(row):
    labels = row['Label'].split('|')
    for label in labels:
        row[label] = 1    
    return row

examples = examples.apply(encoder, axis=1)
examples.fillna(0, inplace=True)

In [8]:
examples.to_csv('../input/image_subset_example/example.csv')

In [ ]:
# define transforms
def get_transform():
    custom_transforms = [Tile(), Rescale((224,224)), 
                         ImageToTensor()]
    return torchvision.transfjorms.Compose(custom_transforms)